In [1]:
module_path = "../"
if findfirst(x -> x == module_path, LOAD_PATH) == nothing
    push!(LOAD_PATH, module_path)
end
using Revise

In [10]:
using Utils
using Solve
using Analysis
using Plots
using DSP
using Optim
using FFTW
using Statistics

In [29]:
B0_new = 0.050 # In Gauss
initial_phases=(pi/2, pi/2)
t_samp = 1e-3 #how often to record data

function display_dw(B1, sim_time)
    sol = run_simulations(sim_time, 1;
        B0=B0_new,
        B1=B1,
        initial_phases=initial_phases,
        saveat=0:t_samp:sim_time,
        )
    p1 = planephase(sol.u[1:3,:,1])
    p2 = planephase(sol.u[4:6,:,1])
    d = mod2pi.((p1 .- p2) .+ pi) .- pi
    plot(sol.t, d)
end

function extract_dw(sol; nsmooth=50)
    # From a solution containing 1 run, extract the frequency difference between
    # neutron and he3 in radians/s
    p1 = planephase(sol.u[1:3,:,1])
    p2 = planephase(sol.u[4:6,:,1])
    d = mod2pi.((p1 .- p2) .+ pi) .- pi #phase difference mod 2pi
    total_d = 0
    total_t = 0

    last_points = zeros(nsmooth)
    for i=1:length(d)-1
        delta_d = d[i+1]-d[i]
        if abs(delta_d) > pi
            continue
        end
        total_d += delta_d
        total_t += (sol.t[i+1]-sol.t[i])
        if length(d)-i <= nsmooth
            last_points[nsmooth+i-length(d)+1] = total_d/total_t
        end
    end
    return mean(last_points)/sol.t[end-nsmooth÷2]
end

function w_diff(B1, sim_time)
    # For a given B1 value, compute frequency difference between
    # neutron and he3 in radians/s
    sol = run_simulations(sim_time, 1;
            B0=B0_new,
            B1=B1,
            initial_phases=initial_phases,
            saveat=0:t_samp:sim_time,
            )
    return extract_dw(sol)
end

w_diff (generic function with 1 method)

In [33]:
sim_time = 100
opt_res = optimize(x->w_diff(x, sim_time)^2, 0.4, 0.5)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [0.400000, 0.500000]
 * Minimizer: 4.028911e-01
 * Minimum: 4.401513e-19
 * Iterations: 10
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 11